In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [22]:
#Directorio.
directorio = r"C:\Users\Juan\Documents\Digital House\Git Digital House\DHDS_IV\midi_stats_v3_drop_cols_.csv"

In [23]:
#Se levanta el dataset.
data = pd.read_csv(directorio)

In [24]:
#Se eliminan los valores nulos.
data = data.fillna(value = 0)

In [25]:
#Se elimina registro que tiende a infinito.
data.drop(data[data.cant_pedales_seg > 9999999].index, inplace = True)

In [26]:
#Redondeo de floats.
for col in data.loc[:, data.dtypes == np.float64].columns:
    data[col] = data[col].apply(lambda x: round(x, 4))

In [27]:
#Visualización de columnas.
pd.options.display.max_columns = 20
#data.head(3)

In [28]:
#Matrices de features y target.
x = data.drop(columns = ["Genero", "tema", "Grupo"], axis = 1)
y = data["Genero"]

In [29]:
#Se importan las clases.
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression

In [30]:
#Se crea la clase para seleccionar las columnas para el procesamiento en los pipelines.
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            return pd.DataFrame(X[self.columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self

In [31]:
#Columnas categóricas / numéricas.
categorical_columns = [col for col in x.columns if x[col].dtypes == 'object']
non_categorical_columns = [col for col in x.columns if col not in categorical_columns]

In [32]:
#Código para el encoder.
categorical_columns = [col for col in x.columns if x[col].dtypes == 'object']
encoder_categories = []
for col in categorical_columns:    
    col_categories = x[col].unique()
    encoder_categories.append(col_categories)

In [33]:
#Pipe para las categóricas.
cs_categorical = ColumnSelector(categorical_columns)
cs_categorical.transform(data).head(3)
categorical_pipe = make_pipeline(ColumnSelector(categorical_columns), OneHotEncoder(categories = encoder_categories, sparse=False))

In [34]:
#Pipe para las numéricas.
cs_non_categorical = ColumnSelector(non_categorical_columns)
cs_non_categorical.transform(data).head(3)
non_categorical_pipe = make_pipeline(ColumnSelector(non_categorical_columns), StandardScaler())

In [35]:
#Unión de los pipes.
union = make_union(categorical_pipe, non_categorical_pipe)

In [36]:
#División del dataset.
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 1)

In [37]:
#Generación del pipeline.
pipeline = Pipeline([('union', union), ('lr', LogisticRegression())])

In [38]:
#Fiteo del pipeline.
pipeline.fit(X_train, y_train)

C:\Users\Juan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('pipeline-1',
                                                 Pipeline(steps=[('columnselector',
                                                                  ColumnSelector(columns=['first_time_signature',
                                                                                          'tonalidad',
                                                                                          'tonalidad_escala',
                                                                                          'info_tracks'])),
                                                                 ('onehotencoder',
                                                                  OneHotEncoder(categories=[array(['3/4', '6/8', '4/4', '3/2', '9/8', '2/4', '12/8', '2/2', '6/4',
       '2/8', '1/4', '1/8', '3/8', '8/4', '9/16', '4/2', '1/...
                                                                                  

In [39]:
#Predicción del pipe en train.
y_pred_train = pipeline.predict(X_train)

In [40]:
#Predicción del pipe en test.
y_pred_test = pipeline.predict(X_test)

In [41]:
#Resultados del modelo.
print(classification_report(y_train, y_pred_train))
print(accuracy_score(y_train, y_pred_train))
print(classification_report(y_test, y_pred_test))
print(accuracy_score(y_test, y_pred_test))

              precision    recall  f1-score   support

 ClassicRock       1.00      1.00      1.00      1520
   Classical       1.00      1.00      1.00      1083
Dance_Techno       1.00      1.00      1.00        16
       Forro       1.00      1.00      1.00         5
       Hymns       1.00      1.00      1.00       242
        Jazz       1.00      1.00      1.00       235
     NEW-AGE       1.00      1.00      1.00       127
        Punk       1.00      1.00      1.00        58
      REGGAE       1.00      1.00      1.00        32
 Samba&Bossa       1.00      1.00      1.00        52
       TANGO       1.00      1.00      1.00        59
      latina       1.00      1.00      1.00       195

    accuracy                           1.00      3624
   macro avg       1.00      1.00      1.00      3624
weighted avg       1.00      1.00      1.00      3624

1.0
              precision    recall  f1-score   support

 ClassicRock       0.95      0.96      0.95       507
   Classical       0